In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import random

In [11]:
STEPS = 50
RANGE = 250
N_PERSONAS = 10

In [12]:
class Persona:
    q_persona = 0
    
    def __init__(self, x_init, y_init, tablero, verbose=False):
        self.numero = Persona.q_persona
        Persona.q_persona += 1
        self.tablero = tablero
        self.x = x_init
        self.y = y_init
        self.verbose = verbose
        
    def get_position(self):
        return self.x, self.y
        
    def step_normal(self):
        movimientos = self.tablero.get_movimientos_posibles(self.x, self.y)
        idx = np.argmax(np.random.uniform(0, 0.4, len(movimientos)))
        self.x, self.y = movimientos[idx]
        if self.verbose:
            print("({}) Posicion x: {}".format(self.numero, self.x))
            print("({}) Posicion y: {}".format(self.numero, self.y))
        return self.x, self.y
        
class Tablero:
    
    def __init__(self, n_dim, n_personas, verbose=False):
        self.verbose = verbose
        self.dim = n_dim
        self.personas = self.__init_personas(n_personas)

    
    def __init_personas(self, n_personas):
        def get_init_positions(n_persons):
            indexes = [(i, j) for i in range(0, self.dim) for j in range(0, self.dim)]
            random.shuffle(indexes)
            return indexes[:n_persons]
        
        def generate_persons(pos):
            persons = []
            n = np.random.rand(len(pos))
            for (x_init, y_init), ni in zip(pos, n):
                if ni <= 0.1:
                    persons.append(PersonaC(x_init, y_init, self, self.verbose))
                elif ni > 0.1 and ni <= 0.3:
                    persons.append(PersonaB(x_init, y_init, self, self.verbose))
                else:
                    persons.append(PersonaA(x_init, y_init, self, self.verbose))
            return persons
        
        init_positions = get_init_positions(n_personas)
        return generate_persons(init_positions)
        
    def get_movimientos_posibles(self, x, y):
        movs = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
        actual_positions = [p.get_position() for p in self.personas]
        possible_movs = set(movs) - set(actual_positions)
        return list(possible_movs)
    
    def siguiente_ronda(self):
        for p in self.personas:
            p.step()
        return self.personas

In [13]:
class PersonaC(Persona):
    def __init__(self, x_inicial, y_inicial, tablero, verbose=False):
        super().__init__(x_inicial, y_inicial, tablero, verbose)
        self.tipo = 'C'
        self.hex_color = '#49111C'
        self.puede_moverse = True
        
    def step(self):
        if self.puede_moverse:
            return self.step_normal()
        return self.x, self.y
    
class PersonaB(Persona):
    def __init__(self, x_inicial, y_inicial, tablero, verbose=False):
        super().__init__(x_inicial, y_inicial, tablero, verbose)
        self.tipo = 'B'
        self.hex_color = '#FF5A5F'
        self.puede_moverse = True
    
    def step(self):
        if self.puede_moverse:
            return self.step_normal()
        return self.x, self.y
    
class PersonaA(Persona):
    def __init__(self, x_inicial, y_inicial, tablero, verbose=False):
        super().__init__(x_inicial, y_inicial, tablero, verbose)
        self.tipo = 'A'
        self.hex_color = '#BFD7EA'
        self.puede_moverse = True
        
    def step(self):
        if self.puede_moverse:
            return self.step_normal()
        return self.x, self.y

In [14]:
def animate_n_persons(n_persons, lines, pos, verbose=False):
    assert n_persons == len(lines)
    assert n_persons == len(pos)
    persons = generate_persons(pos, verbose)
    def animate_persons(i):
        new_lines = []
        for person, line in zip(persons, lines):
            x, y = next(person)
            #line.set_data(person.recorrido_x, person.recorrido_y)
            line.set_data([x], [y])
            line.set_color(person.hex_color)
            new_lines.append(line)
        return new_lines
    return animate_persons

In [15]:
def animate_persons(n_step, tablero, lines):
    new_lines = []
    persons = tablero.siguiente_ronda()
    for person, line in zip(persons, lines):
        x, y = person.get_position()
        line.set_data([x], [y])
        line.set_color(person.hex_color)
        new_lines.append(line)
    return new_lines

In [16]:
def gen_lines(ax, n_personas):
    cmap = plt.cm.get_cmap('hsv', n_personas)
    #lines = [ax.plot([], [], lw=2, color=cmap(n))[0] for n in range(n_personas)]
    #lines = [ax.plot([], [], marker='o', c=cmap(n))[0] for n in range(n_personas)]
    lines = [ax.plot([], [], marker='o')[0] for n in range(n_personas)]
    return lines

In [18]:
#Set up ax
fig = plt.figure(figsize=(21, 10))
ax = plt.axes(xlim=(0, RANGE), ylim=(0, RANGE)) 
lines = gen_lines(ax, N_PERSONAS)
ax.set_title('2D Random Walk', fontsize=22)
ax.set_xlabel('Steps', fontsize=18)
ax.set_ylabel('Value', fontsize=18)
ax.tick_params(labelsize=16)
ax.grid(True, which='major', linestyle='--', color='black', alpha=0.4)

#Set up tablero
tablero = Tablero(RANGE, N_PERSONAS)

# call the animator	 
anim = animation.FuncAnimation(fig, animate_persons, fargs = (tablero, lines),
                               frames=STEPS, interval=295, blit=True)

anim.save('readomwalk.mp4')
plt.close()
"random walk saved"

'random walk saved'